In [1]:
#Package needed for next code block
!pip3 install PyDrive

     |████████████████████████████████| 993kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
#Code to authorize Colab to access Google Drive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Code for reading the data from Google Drive
#Non-augmented train data
download = drive.CreateFile({'id': '1-3R9ZN-yRXLmsm4oGmmuHM_REUwHEen3'})
download.GetContentFile('traindata.npy')

#Augmented train data
download = drive.CreateFile({'id': '1aVjdTfLfkpL5EEAO3MQgXh69XqATCzvI'})
download.GetContentFile('robin_data_0.npy')

download = drive.CreateFile({'id': '1xL0x8M9zmHkjvbAc8gf8R8JKYl_t_Gdu'})
download.GetContentFile('complex_data_0.npy')

download = drive.CreateFile({'id': '1bbwjuDP7lYFDtF11QJ2QswEtLPpYGZYD'})
download.GetContentFile('robin_small.npy')

download = drive.CreateFile({'id': '11b7WwYXKAGE5mt5r6lBPgwZR4K7csKrV'})
download.GetContentFile('robin_train.npy')

download = drive.CreateFile({'id': '1Sjx06gAnJt0onnq5YSRQeKXaVkD5NP9z'})
download.GetContentFile('complex_train.npy')

In [8]:
from __future__ import print_function, division
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Activation
from keras.layers import MaxPooling2D, Concatenate, LeakyReLU, Conv2DTranspose
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
from os import mkdir
from os.path import isdir, abspath

ROBINPATH = abspath("./ROBIN")
COMPLEXPATH = abspath("./Dataset_complex")
#OUTPATH = abspath("./augmented")
train_robin = 'robin_train.npy'
train_complex = 'complex_train.npy'

RESIZE_FACTOR = 32
TRAIN_ON_ROBIN = True
TRAIN_ON_COMPLEX = False

NPY_SAVEFILE = 'traindata.npy'
IMAGE_DIR = 'images/'
LOG_DIR = './logs'

EPOCHS = 1000000
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
DECAY = 0
SAMPLE_INTERVAL = 50


class GAN():
    def __init__(self):
        self.latent_dim = (5, 5)
        #random_file = os.path.join(OUTPATH, os.listdir(OUTPATH)[0])
        self.img_size = np.load(train_robin, allow_pickle=True)[0].shape
        self.img_size += (1,)  # add color channel for conv layers

        #self.this_npy_num_imgs = os.path.join(OUTPATH, os.listdir(OUTPATH)[0])
        self.this_npy_num_imgs = np.load(train_robin, allow_pickle=True)
        self.this_npy_num_imgs = self.this_npy_num_imgs.shape[0]

        optimizer = Adam(LEARNING_RATE, decay=DECAY)

        # Empty any old log directory
        if os.path.exists(LOG_DIR):
            shutil.rmtree(LOG_DIR)
            print("Removed old log directory.")

        os.mkdir(LOG_DIR)
        print('Created new log directory.')

        # Empty any old image directory
        if os.path.exists(IMAGE_DIR):
            shutil.rmtree(IMAGE_DIR)
            print("Removed old image directory.")

        os.mkdir(IMAGE_DIR)
        print('Created new image directory.')

        # Empty the generated image directory
        for the_file in os.listdir(IMAGE_DIR):
            file_path = os.path.join(IMAGE_DIR, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(e)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.summary()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=self.latent_dim)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        n_filts = (32, 16)
        kernel_sizes = (8, 4)

        inp = Input(shape=self.latent_dim)

        layer1 = Flatten()(inp)

        layer1 = Dense(units=n_filts[0] * np.prod(self.img_size))(layer1)
        layer1 = Dropout(rate=0.1)(layer1)

        layer1 = Reshape(target_shape=(self.img_size[0],
                                       self.img_size[1],
                                       n_filts[0]))(layer1)

        for i in range(min(len(n_filts), len(kernel_sizes))):
            layer1 = Conv2DTranspose(filters=n_filts[i],
                                     kernel_size=kernel_sizes[i],
                                     padding="same")(layer1)

        layer1 = Conv2DTranspose(filters=1,
                                 kernel_size=3,
                                 padding="same")(layer1)

        out = Reshape(target_shape=self.img_size)(layer1)
        out = Activation('tanh')(layer1)

        model = Model(inputs=inp, outputs=out)

        # model.summary()

        return model

    def build_discriminator(self):

        d_in = Input(shape=self.img_size)

        d = Conv2D(filters=12,
                   kernel_size=10,
                   activation='relu',
                   padding='same')(d_in)

        d = Dense(units=512, activation='relu')(d)

        d = Dense(units=1, activation='tanh')(d)

        model = Model(inputs=d_in, outputs=d)

        model.summary()

        return model

    def train(self, epochs, batch_size=BATCH_SIZE, sample_interval=50):

        tensorboard = TensorBoard(log_dir=LOG_DIR)
        tensorboard.set_model(self.discriminator)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Detect batch size in npys

            batch_size = min(self.this_npy_num_imgs, batch_size)

            idx = np.random.randint(0, self.this_npy_num_imgs-1, batch_size)

            # Select a random batch of images
            #self.X_train = os.path.join(OUTPATH, np.random.choice(os.listdir(OUTPATH)))
            self.X_train = np.load(train_robin, allow_pickle=True)
            self.X_train = self.X_train[idx]
            self.X_train = np.expand_dims(self.X_train, axis=3)
            self.X_train = self.X_train / (255/2) - 1

            noise = np.random.normal(-1, 1, ((batch_size,) + self.latent_dim))

            # Adversarial ground truths
            valid = np.ones(self.X_train.shape)
            fake = np.zeros(self.X_train.shape)

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            if epoch == 0 or accuracy < 80:
                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(self.X_train,
                                                                valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            else:
                # Test the discriminator
                d_loss_real = self.discriminator.test_on_batch(self.X_train,
                                                               valid)
                d_loss_fake = self.discriminator.test_on_batch(gen_imgs, fake)

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            accuracy = 100*d_loss[1]

            # ---------------------
            #  Train Generator
            # ---------------------
            noise = np.random.normal(-1, 1, ((batch_size,) + self.latent_dim))

            if epoch == 0 or accuracy > 20:
                # Train the generator (to have the discriminator label samples
                # as valid)
                g_loss = self.combined.train_on_batch(noise, valid)
            else:
                # Train the generator (to have the discriminator label samples
                # as valid)
                g_loss = self.combined.test_on_batch(noise, valid)

            tensorboard.on_epoch_end(epoch, {'generator loss': g_loss,
                                             'discriminator loss': d_loss[0],
                                             'Accuracy': accuracy,
                                             'Comb. loss': g_loss + d_loss[0]})

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                print(f"@ {epoch:{len(str(EPOCHS))}}:\t"
                      f"Accuracy: {int(accuracy):3}%\t"
                      f"G-Loss: {g_loss:6.3f}\t"
                      f"D-Loss: {d_loss[0]:6.3f}\t"
                      f"Combined: {g_loss+d_loss[0]:6.3f}")
                self.sample_images(epoch)

        tensorboard.on_train_end(tensorboard)
        self.discriminator.save('discriminator.h5')
        self.generator.save('generator.h5')

    def sample_images(self, epoch):
        r = 3
        noise = np.random.normal(-1, 1, ((r,) + self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Select a random image
        #real_imgs = os.path.join(OUTPATH, np.random.choice(os.listdir(OUTPATH)))
        real_imgs = np.load(train_robin, allow_pickle=True)
        real_imgs = real_imgs[np.random.randint(0, BATCH_SIZE*8), :, :]

        fig, axs = plt.subplots(r)

        axs[0].imshow(real_imgs, cmap='gray')
        axs[0].axis('off')

        cnt = 0
        for i in range(r-1):
            axs[i+1].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i+1].axis('off')
            cnt += 1

        fig.savefig(IMAGE_DIR+"%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    # Train the GAN
    gan = GAN()
    gan.train(epochs=EPOCHS, sample_interval=SAMPLE_INTERVAL)


Removed old log directory.
Created new log directory.
Removed old image directory.
Created new image directory.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 230, 230, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 230, 230, 12)      1212      
_________________________________________________________________
dense_12 (Dense)             (None, 230, 230, 512)     6656      
_________________________________________________________________
dense_13 (Dense)             (None, 230, 230, 1)       513       
Total params: 8,381
Trainable params: 8,381
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)        

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


@       0:	Accuracy:  50%	G-Loss: 14.493	D-Loss:  8.053	Combined: 22.546


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


@      50:	Accuracy:  50%	G-Loss:  4.915	D-Loss:  7.912	Combined: 12.827
@     100:	Accuracy:  50%	G-Loss:  3.948	D-Loss:  1.768	Combined:  5.716
@     150:	Accuracy:  50%	G-Loss:  3.773	D-Loss:  1.499	Combined:  5.271
@     200:	Accuracy:  50%	G-Loss:  3.698	D-Loss:  1.343	Combined:  5.041
@     250:	Accuracy:  50%	G-Loss:  3.606	D-Loss:  1.207	Combined:  4.813
@     300:	Accuracy:  50%	G-Loss:  3.497	D-Loss:  1.062	Combined:  4.559
@     350:	Accuracy:  50%	G-Loss:  3.433	D-Loss:  0.922	Combined:  4.355
@     400:	Accuracy:  50%	G-Loss:  3.308	D-Loss:  0.800	Combined:  4.108
@     450:	Accuracy:  50%	G-Loss:  3.200	D-Loss:  0.705	Combined:  3.906
@     500:	Accuracy:  50%	G-Loss:  3.013	D-Loss:  0.641	Combined:  3.653
@     550:	Accuracy:  50%	G-Loss:  2.737	D-Loss:  0.585	Combined:  3.322
@     600:	Accuracy:  50%	G-Loss:  1.959	D-Loss:  0.574	Combined:  2.533
@     650:	Accuracy:  50%	G-Loss:  1.657	D-Loss:  0.592	Combined:  2.249
@     700:	Accuracy:  50%	G-Loss:  1.154	D-Loss:  0

KeyboardInterrupt: ignored